In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

import os
from dotenv import load_dotenv
import boto3

In [2]:
# Carregar as variáveis de ambiente do arquivo .secrets
load_dotenv('.secrets')

# Definir a configuração do backend store do MLflow
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('aws_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('aws_secret_key')
os.environ['MLFLOW_S3_IGNORE_TLS'] = 'true'
os.environ['MLFLOW_TRACKING_URI'] = os.getenv('bucket_name')

s3_client = boto3.client('s3')

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

2023/07/04 16:12:13 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-weslley/1', creation_time=1688497933267, experiment_id='1', last_update_time=1688497933267, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


/home/wa/miniconda3/envs/venv_mlengineer/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/home/wa/miniconda3/envs/venv_mlengineer/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mlflow-weslley.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/wa/miniconda3/envs/venv_mlengineer/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mlflow-weslley.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/wa/miniconda3/envs/venv_mlengineer/lib/python3.8/site-packages/urllib3/connecti

In [7]:
from mlflow.tracking import MlflowClient

In [8]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '3bdf3831eac4453591e615658d3eacd1'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [9]:
path = mlflow.artifacts.download_artifacts(
    run_id=RUN_ID,
    artifact_path='model/model.pkl',
)

/home/wa/miniconda3/envs/venv_mlengineer/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mlflow-weslley.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/wa/miniconda3/envs/venv_mlengineer/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mlflow-weslley.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/wa/miniconda3/envs/venv_mlengineer/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mlflow-weslley.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthe

In [10]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [11]:
dv.get_params()

{'memory': None,
 'steps': [('dictvectorizer', DictVectorizer()),
  ('randomforestregressor',
   RandomForestRegressor(max_depth=20, min_samples_leaf=10, n_jobs=-1,
                         random_state=0))],
 'verbose': False,
 'dictvectorizer': DictVectorizer(),
 'randomforestregressor': RandomForestRegressor(max_depth=20, min_samples_leaf=10, n_jobs=-1,
                       random_state=0),
 'dictvectorizer__dtype': numpy.float64,
 'dictvectorizer__separator': '=',
 'dictvectorizer__sort': True,
 'dictvectorizer__sparse': True,
 'randomforestregressor__bootstrap': True,
 'randomforestregressor__ccp_alpha': 0.0,
 'randomforestregressor__criterion': 'squared_error',
 'randomforestregressor__max_depth': 20,
 'randomforestregressor__max_features': 'auto',
 'randomforestregressor__max_leaf_nodes': None,
 'randomforestregressor__max_samples': None,
 'randomforestregressor__min_impurity_decrease': 0.0,
 'randomforestregressor__min_samples_leaf': 10,
 'randomforestregressor__min_samples_s